In [9]:
import socket
import time
import csv


def send_data(speed1, speed2, direction1, direction2, IP, PORT = 80):
    # combined_value = (speed1 << 16) | speed2 
    combined_value = (speed1 << 24) | (speed2 << 16) | (direction1 << 8) | direction2
    retry_delay = 2  # Tempo de espera (em segundos) entre tentativas

    while True:  # Loop infinito para tentativas indefinidas
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                print("Trying to connect to ESP32...")
                s.connect((IP, PORT))
                s.sendall(combined_value.to_bytes(4, byteorder='little'))  # Envia o valor como 4 bytes
                print("Data sent successfully.")
                return  # Sai da função após o envio bem-sucedido
        except ConnectionRefusedError:
            print(f"Connection refused, retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)  # Aguarda antes de tentar novamente


## ENVIA DADOS DE VELOCIDADE E PID

In [1]:
import socket
import time

def send_speed(speed_left, speed_right, dir_left, dir_right, ip, port=80):
    """
    Envia dados binários codificados como 4 bytes:
    [speed_left << 24 | speed_right << 16 | dir_left << 8 | dir_right]
    """
    value = (speed_left << 24) | (speed_right << 16) | (dir_left << 8) | dir_right

    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(3)
            # Desabilita o algoritmo de Nagle pra mandar tudo de uma vez
            s.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
            s.connect((ip, port))
            # Pequeno atraso para garantir handshake concluído
            time.sleep(0.01)
            # Envia em little-endian (conforme sua leitura no ESP32)
            s.sendall(value.to_bytes(4, byteorder='little'))
            s.shutdown(socket.SHUT_WR)
            print("✔️ Velocidade enviada com sucesso.")
    except Exception as e:
        print("❌ Falha ao enviar velocidade:", e)

def send_pid_param(param_str, ip, port=80):
    """
    Envia parâmetros PID como texto (ex: "KPR=1.23\n").
    """
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(3)
            s.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
            s.connect((ip, port))
            time.sleep(0.01)
            s.sendall((param_str.strip() + "\n").encode())
            s.shutdown(socket.SHUT_WR)
            print(f"✔️ Parâmetro enviado: {param_str}")
    except Exception as e:
        print("❌ Falha ao enviar parâmetro:", e)


### Exemplos de usos:

## Utilização

In [2]:
from configs.config import IP_ARES, IP_ZEUS, IP_KRATOS

#### IPs:

In [3]:
IP_ROBO = IP_ARES

In [4]:
IP_ROBO = IP_ZEUS


In [3]:
IP_ROBO = IP_KRATOS

In [6]:
IP_ROBO = "192.168.209.30"

#### Velocidades:

In [10]:

send_speed(177, 150, 1, 1, IP_ROBO)

❌ Falha ao enviar velocidade: timed out


In [16]:


send_speed(0, 0, 1, 1, IP_ROBO)

✔️ Velocidade enviada com sucesso.
